In [1]:
import pandas as pd

df_loaded = pd.read_pickle('experiment_results.pkl')

In [2]:
df_loaded

,experiment_id,timestamp,function,optimizer,learning_rate,start_point,start_x,start_y,final_point,final_x,final_y,final_value,iterations,runtime,converged,path,path_array
0,1,2025-09-17T21:32:40.160575,convex,gd,0.0010,"(-2, 2)",-2.0,2.0,"[-0.03648485044750143, 0.03648485044750143]",-0.036485,0.036485,0.002673,1999,0.005269,True,"[[-2.0, 2.0], [-1.996, 1.996], [-1.992008, 1.9...","[[-2.0, 2.0], [-1.996, 1.996], [-1.992008, 1.9..."
1,2,2025-09-17T21:32:40.161885,convex,gd,0.0100,"(-2, 2)",-2.0,2.0,"[-0.0035157482001922508, 0.0035157482001922508]",-0.003516,0.003516,0.000025,314,0.000795,True,"[[-2.0, 2.0], [-1.96, 1.96], [-1.9208, 1.9208]...","[[-2.0, 2.0], [-1.96, 1.96], [-1.9208, 1.9208]..."
2,3,2025-09-17T21:32:40.162036,convex,gd,0.1000,"(-2, 2)",-2.0,2.0,"[-0.0010141204801825839, 0.0010141204801825839]",-0.001014,0.001014,0.000002,34,0.000098,True,"[[-2.0, 2.0], [-1.6, 1.6], [-1.28, 1.28], [-1....","[[-2.0, 2.0], [-1.6, 1.6], [-1.28, 1.28], [-1...."
3,4,2025-09-17T21:32:40.162253,convex,newton,0.5000,"(-2, 2)",-2.0,2.0,"[0.0, 0.0]",0.000000,0.000000,0.000000,1,0.000192,True,"[[-2.0, 2.0], [0.0, 0.0]]","[[-2.0, 2.0], [0.0, 0.0]]"
4,5,2025-09-17T21:32:40.162312,convex,newton,1.0000,"(-2, 2)",-2.0,2.0,"[0.0, 0.0]",0.000000,0.000000,0.000000,1,0.000035,True,"[[-2.0, 2.0], [0.0, 0.0]]","[[-2.0, 2.0], [0.0, 0.0]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,104,2025-09-17T21:32:40.832725,multimodal,adagrad,0.1000,"(3, 3)",3.0,3.0,"[2.5963247317036813, 2.5963247317036813]",2.596325,2.596325,-3.617963,41,0.000350,True,"[[3.0, 3.0], [2.900000000217922, 2.90000000021...","[[3.0, 3.0], [2.900000000217922, 2.90000000021..."
104,105,2025-09-17T21:32:40.832832,multimodal,adagrad,1.0000,"(3, 3)",3.0,3.0,"[2.5955031490063276, 2.5955031490063276]",2.595503,2.595503,-3.617966,11,0.000089,True,"[[3.0, 3.0], [2.000000002179219, 2.00000000217...","[[3.0, 3.0], [2.000000002179219, 2.00000000217..."
105,106,2025-09-17T21:32:40.876316,multimodal,adam,0.0001,"(3, 3)",3.0,3.0,"[2.822612482552649, 2.822612482552649]",2.822612,2.822612,-3.056440,1999,0.042884,True,"[[3.0, 3.0], [2.999900000000218, 2.99990000000...","[[3.0, 3.0], [2.999900000000218, 2.99990000000..."
106,107,2025-09-17T21:32:40.898212,multimodal,adam,0.0010,"(3, 3)",3.0,3.0,"[2.5981923839170564, 2.5981923839170564]",2.598192,2.598192,-3.617903,1041,0.020394,True,"[[3.0, 3.0], [2.9990000000021793, 2.9990000000...","[[3.0, 3.0], [2.9990000000021793, 2.9990000000..."
